# Analyze A/B Test Results 

This project will assure you have mastered the subjects covered in the statistics lessons. We have organized the current notebook into the following sections: 

- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)
- [Final Check](#finalcheck)
- [Submission](#submission)

Specific programming tasks are marked with a **ToDo** tag. 

<a id='intro'></a>
## Introduction

A/B tests are very commonly performed by data analysts and data scientists. For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should:
- Implement the new webpage, 
- Keep the old webpage, or 
- Perhaps run the experiment longer to make their decision.

Each **ToDo** task below has an associated quiz present in the classroom.  Though the classroom quizzes are **not necessary** to complete the project, they help ensure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the [rubric](https://review.udacity.com/#!/rubrics/1214/view) specification. 

<a id='probability'></a>
## Part I - Probability

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
random.seed(42)

### ToDo 1.1
Now, read in the `ab_data.csv` data. Store it in `df`. Below is the description of the data, there are a total of 5 columns:

<center>

|Data columns|Purpose|Valid values|
| ------------- |:-------------| -----:|
|user_id|Unique ID|Int64 values|
|timestamp|Time stamp when the user visited the webpage|-|
|group|In the current A/B experiment, the users are categorized into two broad groups. <br>The `control` group users are expected to be served with `old_page`; and `treatment` group users are matched with the `new_page`. <br>However, **some inaccurate rows** are present in the initial data, such as a `control` group user is matched with a `new_page`. |`['control', 'treatment']`|
|landing_page|It denotes whether the user visited the old or new webpage.|`['old_page', 'new_page']`|
|converted|It denotes whether the user decided to pay for the company's product. Here, `1` means yes, the user bought the product.|`[0, 1]`|
</center>
Use your dataframe to answer the questions in Quiz 1 of the classroom.

**a.** Read in the dataset from the `ab_data.csv` file and take a look at the top few rows here:

In [ ]:
df = pd.read_csv('ab_data.csv')
df.head()

**b.** Use the cell below to find the number of rows in the dataset.

In [ ]:
df.shape[0]

**c.** The number of unique users in the dataset.

In [ ]:
df['user_id'].nunique()

**d.** The proportion of users converted.

In [ ]:
df['converted'].mean()

**e.** The number of times when the "group" is `treatment` but "landing_page" is not a `new_page`.

In [ ]:
treat_old = df.query('group == "treatment" & landing_page != "new_page"').shape[0]
control_new = df.query('group == "control" & landing_page != "old_page"').shape[0]

unmatching_rows = treat_old + control_new

unmatching_rows

**f.** Do any of the rows have missing values?

In [ ]:
df.info()

### ToDo 1.2  
In a particular row, the **group** and **landing_page** columns should have either of the following acceptable values:

|user_id| timestamp|group|landing_page|converted|
|---|---|---|---|---|
|XXXX|XXXX|`control`| `old_page`|X |
|XXXX|XXXX|`treatment`|`new_page`|X |


It means, the `control` group users should match with `old_page`; and `treatment` group users should matched with the `new_page`. 

However, for the rows where `treatment` does not match with `new_page` or `control` does not match with `old_page`, we cannot be sure if such rows truly received the new or old wepage.  


Use **Quiz 2** in the classroom to figure out how should we handle the rows where the group and landing_page columns don't match?

**a.** Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [ ]:
# Remove the inaccurate rows, and store the result in a new dataframe df2

df2 = df.drop(df[(df['group'] == 'control') & (df['landing_page'] != 'old_page')].index)
df2 = df2.drop(df2[(df2['group'] == 'treatment') & (df['landing_page'] != 'new_page')].index)

In [ ]:
# Double Check all of the incorrect rows were removed from df2 - 
# Output of the statement below should be 0

df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

### ToDo 1.3  
Use **df2** and the cells below to answer questions for **Quiz 3** in the classroom.

**a.** How many unique **user_id**s are in **df2**?

In [ ]:
df2['user_id'].nunique()

**b.** There is one **user_id** repeated in **df2**.  What is it?

In [ ]:
df2[df2['user_id'].duplicated()]['user_id']

**c.** Display the rows for the duplicate **user_id**? 

In [ ]:
df2[df2['user_id'] == 773192]

**d.** Remove **one** of the rows with a duplicate **user_id**, from the **df2** dataframe.

In [ ]:
# Remove one of the rows with a duplicate user_id..
df2.drop(2893, axis=0, inplace=True)

# Check again if the row with a duplicate user_id is deleted or not
df2[df2['user_id'] == 773192]

### ToDo 1.4  
Use **df2** in the cells below to answer the quiz questions related to **Quiz 4** in the classroom.

**a.** What is the probability of an individual converting regardless of the page they receive?<br><br>

In [ ]:
df2['converted'].mean()

**b.** Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
cont_conv_p = df2.query('group == "control"')['converted'].mean()
cont_conv_p

**c.** Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
treat_conv_p = df2.query('group == "treatment"')['converted'].mean()
treat_conv_p

In [ ]:
# Calculate the actual difference (obs_diff) between the conversion rates for the two groups.
obs_diff = treat_conv_p - cont_conv_p
obs_diff

**d.** What is the probability that an individual received the new page?

In [ ]:
len(df2[df2['landing_page'] == 'new_page'])/len(df2)

**e.** Consider your results from parts (a) through (d) above, and explain below whether the new `treatment` group users lead to more conversions.

>**The treatment group doesn't lead to more conversions, as the control group had a slightly higher probability of conversion than the treatment group.**



<a id='ab_test'></a>
## Part II - A/B Test

Since a timestamp is associated with each event, you could run a hypothesis test continuously as long as you observe the events. 

However, then the hard questions would be: 
- Do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  
- How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


### ToDo 2.1
For now, consider you need to make the decision just based on all the data provided.  

> Recall that you just calculated that the "converted" probability (or rate) for the old page is *slightly* higher than that of the new page (ToDo 1.4.c). 

If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should be your null and alternative hypotheses (**$H_0$** and **$H_1$**)?  

You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the "converted" probability (or rate) for the old and new pages respectively.



$$H_0: p_{new} - p_{old} \leq 0$$

$$H_1: p_{new} - p_{old} > 0$$


### ToDo 2.2 - Null Hypothesis $H_0$ Testing
Under the null hypothesis $H_0$, assume that $p_{new}$ and $p_{old}$ are equal. Furthermore, assume that $p_{new}$ and $p_{old}$ both are equal to the **converted** success rate in the `df2` data regardless of the page. So, our assumption is: <br><br>
<center>
$p_{new}$ = $p_{old}$ = $p_{population}$
</center>

In this section, you will: 

- Simulate (bootstrap) sample data set for both groups, and compute the  "converted" probability $p$ for those samples. 


- Use a sample size for each group equal to the ones in the `df2` data.


- Compute the difference in the "converted" probability for the two samples above. 


- Perform the sampling distribution for the "difference in the converted probability" between the two simulated-samples over 10,000 iterations; and calculate an estimate. 



Use the cells below to provide the necessary parts of this simulation.  You can use **Quiz 5** in the classroom to make sure you are on the right track.



**a.** What is the **conversion rate** for $p_{new}$ under the null hypothesis? 

In [ ]:
p_new = df2['converted'].mean()
p_new

**b.** What is the **conversion rate** for $p_{old}$ under the null hypothesis? 

In [ ]:
p_old = df2['converted'].mean()
p_old

**c.** What is $n_{new}$, the number of individuals in the treatment group? <br><br>


In [ ]:
n_new = df2[df2['group'] == 'treatment'].shape[0]
n_new

**d.** What is $n_{old}$, the number of individuals in the control group?

In [ ]:
n_old = df2[df2['group'] == 'control'].shape[0]
n_old

**e. Simulate Sample for the `treatment` Group**<br> 
Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null hypothesis.  <br><br>


In [ ]:
# Simulate a Sample for the treatment Group

new_page_converted = np.random.choice([0,1], size=n_new, p=[(1-p_new), p_new])

**f. Simulate Sample for the `control` Group** <br>
Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null hypothesis. <br> Store these $n_{old}$ 1's and 0's in the `old_page_converted` numpy array.


In [ ]:
# Simulate a Sample for the control Group

old_page_converted = np.random.choice([0,1], size=n_old, p=[(1-p_old), p_old])

**g.** Find the difference in the "converted" probability $(p{'}_{new}$ - $p{'}_{old})$ for your simulated samples from the parts (e) and (f) above. 



In [ ]:
diff = new_page_converted.mean() - old_page_converted.mean()
diff


**h. Sampling distribution** <br>
Re-create `new_page_converted` and `old_page_converted` and find the $(p{'}_{new}$ - $p{'}_{old})$ value 10,000 times using the same simulation process you used in parts (a) through (g) above. 

<br>
Store all  $(p{'}_{new}$ - $p{'}_{old})$  values in a NumPy array called `p_diffs`.


In [ ]:
# Sampling distribution 

new_page_converted = np.random.binomial(n_new, p_new, 10000)/n_new
old_page_converted = np.random.binomial(n_old, p_old, 10000)/n_old
p_diffs = new_page_converted - old_page_converted

**i. Histogram**<br> 
Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.<br><br>

Also, use `plt.axvline()` method to mark the actual difference observed  in the `df2` data (recall `obs_diff`), in the chart.  

In [ ]:
p_diffs = np.array(p_diffs)

plt.hist(p_diffs);
plt.axvline(obs_diff, color='r', linewidth=2);
plt.title('Sampling distribution according to the null hypothesis');
plt.xlabel('Difference between the conversion probabilities of old and new page');
plt.ylabel('Frequency in the sampling distribution');  

**j.** What proportion of the **p_diffs** are greater than the actual difference observed in the `df2` data?

In [ ]:
p_value = (p_diffs > obs_diff).mean()
p_value

**k.** Please explain in words what you have just computed in part **j** above.  
 - What is this value called in scientific studies?  
 - What does this value signify in terms of whether or not there is a difference between the new and old pages? *Hint*: Compare the value above with the "Type I error rate (0.05)". 

>**1. This value is called a "p-value".**

>**2. Since the p-value is larger than the Type I error rate of 5%, This suggests that the sample came from a population where we fail to reject the null hypothesis, which means that in the population, there is no difference in the conversion rates between the old and new pages, or that the old page is even better than the new one.**

**l. Using Built-in Methods for Hypothesis Testing**<br>
We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. 

Fill in the statements below to calculate the:
- `convert_old`: number of conversions with the old_page
- `convert_new`: number of conversions with the new_page
- `n_old`: number of individuals who were shown the old_page
- `n_new`: number of individuals who were shown the new_page


In [ ]:
import statsmodels.api as sm

# number of conversions with the old_page
convert_old = df2.query('group == "control"')['converted'].sum()

# number of conversions with the new_page
convert_new = df2.query('group == "treatment"')['converted'].sum()

# number of individuals who were shown the old_page
n_old = df2.query('group == "control"').shape[0]

# number of individuals who received new_page
n_new = df2.query('group == "treatment"').shape[0]

**m.** Now use `sm.stats.proportions_ztest()` to compute your test statistic and p-value.  [Here](https://www.statsmodels.org/stable/generated/statsmodels.stats.proportion.proportions_ztest.html) is a helpful link on using the built in.

The syntax is: 
```bash
proportions_ztest(count_array, nobs_array, alternative='larger')
```
where, 
- `count_array` = represents the number of "converted" for each group
- `nobs_array` = represents the total number of observations (rows) in each group
- `alternative` = choose one of the values from `[‘two-sided’, ‘smaller’, ‘larger’]` depending upon two-tailed, left-tailed, or right-tailed respectively. 


The built-in function above will return the z_score, p_value. 

---
### About the two-sample z-test
Recall that you have plotted a distribution `p_diffs` representing the
difference in the "converted" probability  $(p{'}_{new}-p{'}_{old})$  for your two simulated samples 10,000 times. 

Another way for comparing the mean of two independent and normal distribution is a **two-sample z-test**. You can perform the Z-test to calculate the Z_score, as shown in the equation below:

$$
Z_{score} = \frac{ (p{'}_{new}-p{'}_{old}) - (p_{new}  -  p_{old})}{ \sqrt{ \frac{\sigma^{2}_{new} }{n_{new}} + \frac{\sigma^{2}_{old} }{n_{old}}  } }
$$

where,
- $p{'}$ is the "converted" success rate in the sample
- $p_{new}$ and $p_{old}$ are the "converted" success rate for the two groups in the population. 
- $\sigma_{new}$ and $\sigma_{new}$ are the standard deviation for the two groups in the population. 
- $n_{new}$ and $n_{old}$ represent the size of the two groups or samples (it's same in our case)


>Z-test is performed when the sample size is large, and the population variance is known. The z-score represents the distance between the two "converted" success rates in terms of the standard error. 

Next step is to make a decision to reject or fail to reject the null hypothesis based on comparing these two values: 
- $Z_{score}$
- $Z_{\alpha}$ or $Z_{0.05}$, also known as critical value at 95% confidence interval.  $Z_{0.05}$ is 1.645 for one-tailed tests,  and 1.960 for two-tailed test. You can determine the $Z_{\alpha}$ from the z-table manually. 

Decide if your hypothesis is either a two-tailed, left-tailed, or right-tailed test. Accordingly, reject OR fail to reject the  null based on the comparison between $Z_{score}$ and $Z_{\alpha}$. We determine whether or not the $Z_{score}$ lies in the "rejection region" in the distribution. In other words, a "rejection region" is an interval where the null hypothesis is rejected iff the $Z_{score}$ lies in that region.




Reference: 
- Example 9.1.2 on this [page](https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/09%3A_Two-Sample_Problems/9.01%3A_Comparison_of_Two_Population_Means-_Large_Independent_Samples), courtesy www.stats.libretexts.org

---




In [ ]:
import statsmodels.api as sm

# ToDo: Complete the sm.stats.proportions_ztest() method arguments

z_score, p_value = sm.stats.proportions_ztest(np.array([convert_new, convert_old]), 
                                              np.array([n_new, n_old]),
                                              alternative = 'larger')

print(z_score, p_value)

**n.** What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?<br><br>


>**This is a right-tailed test with a z-alpha of 1.645, and the z-score means that the observed difference in the "converted" probability is 1.3109 standard deviations below the null mean (0), so we can conclude that since -1.3109 < 1.645, we fail to reject the null hypothesis just like our previous steps in parts j-k. As for the p-value, it is about the same as what I've computed before, and also means the same thing.**

<a id='regression'></a>
### Part III - A regression approach

### ToDo 3.1 
In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br><br> 

**a.** Since each row in the `df2` data is either a conversion or no conversion, what type of regression should you be performing in this case?

>**I should be performing Logistic Regression, since conversion is a categorical variable represented by 0 for 'no', and 1 for 'yes'.**

**b.** The goal is to use **statsmodels** library to fit the regression model you specified in part **a.** above to see if there is a significant difference in conversion based on the page-type a customer receives. However, you first need to create the following two columns in the `df2` dataframe:
 1. `intercept` - It should be `1` in the entire column. 
 2. `ab_page` - It's a dummy variable column, having a value `1` when an individual receives the **treatment**, otherwise `0`.  

In [ ]:
df2['intercept'] = 1

df2[['old_page', 'ab_page']] = pd.get_dummies(df2['group'])
df2.drop('old_page', axis=1, inplace=True)

df2.head()

**c.** Use **statsmodels** to instantiate your regression model on the two columns you created in part (b). above, then fit the model to predict whether or not an individual converts. 


In [ ]:
logit_mod = sm.Logit(df2['converted'], df2[['intercept', 'ab_page']])
results = logit_mod.fit()

**d.** Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results.summary2()

**e.** What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?<br><br>  



>**The p-value for ab_page is 0.1899, and it is different from the previous p-value because they concern different null hypotheses from each other.**

>**The two-sided hypotheses for this regression model is:**

$H_0: \beta_{1} = 0$ **------->** *"The page shown is NOT statistically significant in predicting whether a website visitor buys the product."*

$H_1: \beta_{1} \neq 0$ **------->** *"The page shown IS statistically significant in predicting whether a website visitor buys the product."*



>**While the one-sided hypotheses for the previous method was:**

$H_0: p_{new} - p_{old} \leq 0$ **------->** *"The conversion rate for the new page is equal to or less than the old page (the old page is better)."*

$H_1: p_{new} - p_{old} > 0$ **------->** *"The conversion rate for the new page is more than the old page (the new page is better)."*



>**Now, 0.1899 is more then the Type I error rate of 0.05, and so we fail to reject the regression model null hypothesis - meaning that the coefficient for the ab_page is 0 and it doesn't have an effect on whether a user buys or not .**

**f.** Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

>**It'd be a good idea to consider other factors because the page-type variable is not statiscally significant for predicting user conversion, and I don't think there are disadvantages to doing that in this case since it won't result in an overfit model.**

**g. Adding countries**<br> 
Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. 

1. You will need to read in the **countries.csv** dataset and merge together your `df2` datasets on the appropriate rows. You call the resulting dataframe `df_merged`. [Here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) are the docs for joining tables. 

2. Does it appear that country had an impact on conversion?  To answer this question, consider the three unique values, `['UK', 'US', 'CA']`, in the `country` column. Create dummy variables for these country columns. 

 Provide the statistical output as well as a written response to answer this question.

In [ ]:
# Read the countries.csv

countries = pd.read_csv('countries.csv')
countries.head()

In [ ]:
# Join with the df2 dataframe

df_merged = df2.join(countries.set_index('user_id'), on='user_id')
df_merged.head()

In [ ]:
# Create the necessary dummy variables

df_merged[['CA', 'UK', 'US']] = pd.get_dummies(df_merged['country'])
df_merged.head()

In [ ]:
logit_mod = sm.Logit(df_merged['converted'], df_merged[['intercept', 'ab_page', 'CA', 'UK']])
results = logit_mod.fit()

In [ ]:
results.summary2()

>**Before looking at the coefficients, I first looked at the p-values compared to the Type I error rate. I found that the p-values for the countries exceeded the 0.05 error rate, so that means they are NOT statistically significant regarding user conversion according to the null hypothesis.**

**h. Fit your model and obtain the results**<br> 
Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if are there significant effects on conversion.  **Create the necessary additional columns, and fit the new model.** 


Provide the summary results (statistical output), and your conclusions (written response) based on the results. 

In [ ]:
df_merged['CA*ab_page'] = df_merged['CA'] * df_merged['ab_page']
df_merged['UK*ab_page'] = df_merged['UK'] * df_merged['ab_page']
df_merged['US*ab_page'] = df_merged['US'] * df_merged['ab_page']

df_merged.head()

In [ ]:
# Fit your model, and summarize the results

logit_mod = sm.Logit(df_merged['converted'], df_merged[['intercept', 'ab_page', 'CA', 'UK', 'CA*ab_page', 'UK*ab_page']])
results = logit_mod.fit()

results.summary2()

>**The p-values for the interaction terms are all more than the 0.05 error rate, suggesting they have no statistical effect on user conversion. We fail to reject the null hypotheses that states their coefficients = 0.**

In [ ]:
df_merged['timestamp'] = pd.to_datetime(df_merged['timestamp'])

df_merged['timestamp'].max() - df_merged['timestamp'].min()

>**Above, I calculated the time period over which the experiment was run, then I googled how much time does an A/B test needs in order to give representative results, and 3 weeks seems enough for users to interact with the new webpage.**

>**My final conclusion would be to keep the old page, since my p-values from part II suggest a high probability that the data came from a population where the old page conversion probability is equal to, or even worse than, the new page, and I fail to reject the null hypothesis.**

>**As for the regression model, I couldn't find any statistically significant relations between any variable and the conversion, judging by the high p-values.**

<a id='finalcheck'></a>
## Final Check!

Congratulations!  You have reached the end of the A/B Test Results project!  You should be very proud of all you have accomplished!


<a id='submission'></a>
## Submission
You may either submit your notebook through the "SUBMIT PROJECT" button at the bottom of this workspace, or you may work from your local machine and submit on  the last page of this project lesson.  

1. Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).


2. Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.


3. Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])